In [2]:
import pandas as pd
import numpy as np
import random

# File operations
from copy import deepcopy
import glob
import os
from pathlib import Path

# Stats
from scipy.spatial.distance import cdist
from scipy.stats import spearmanr, wilcoxon, mannwhitneyu
import statsmodels.stats.multitest

# Neuromaps
import neuromaps
from neuromaps.parcellate import Parcellater
from neuromaps import datasets, images, nulls, resampling, stats
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation, fetch_fslr
from enigmatoolbox.datasets import load_sc, load_fc
from nilearn.image import resample_to_img

# Set seed to 127
random.seed(127)

# Add connectome workbench to path
os.environ['PATH'] = os.environ['PATH'] + ':/Applications/workbench/bin_macosx64'

%load_ext rpy2.ipython

In [3]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(cowplot)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

In [10]:
#### GROUP-AVERAGED FUNCTIONAL CONNECTOME FROM ENIGMA #### 
# Load cortico-cortical functional connectivity data
fc_ctx, fc_ctx_labels, _, _ = load_fc(parcellation='aparc')

# Read in preprocessed Pearson-based functional connectivity data from ENIGMA
HCP_avg_Pearson_FC = (pd.DataFrame(fc_ctx, columns=fc_ctx_labels)
                      .assign(Brain_Region_From = fc_ctx_labels)
                      .melt(id_vars=["Brain_Region_From"], 
                             var_name="Brain_Region_To", 
                             value_name="Mean_Pearson_R")
                      .assign(Base_Region_From = lambda x: x['Brain_Region_From'].str.replace("L_", "").str.replace("R_", ""),
                              Base_Region_To = lambda x: x['Brain_Region_To'].str.replace("L_", "").str.replace("R_", ""),
                              Hemi_From = lambda x: np.where(x['Brain_Region_From'].str.startswith("L_"), "Left", "Right"),
                              Hemi_To = lambda x: np.where(x['Brain_Region_To'].str.startswith("L_"), "Left", "Right"))
)
pearson_homotopic_res_mean = (HCP_avg_Pearson_FC
                              .query("Base_Region_From == Base_Region_To")
                              .groupby(["Base_Region_From"])['Mean_Pearson_R']
                              .mean() 
                              .reset_index()
                              .rename(columns={"Base_Region_From": "Base_Region"})
                              )

ENIGMA_sc_ctx, ENIGMA_sc_ctx_labels, _, _ = load_sc(parcellation='aparc')

fallon2020_SC_data = (pd
                      .read_csv("/Users/abry4213/data/HCP/HCP100/raw_data/diffusion_MRI/aparc_HCP100_group_avg_SIFT2_mean.csv", header=None, names=ENIGMA_sc_ctx_labels)
                      .assign(Brain_Region_To = ENIGMA_sc_ctx_labels)
                      .melt(id_vars='Brain_Region_To', var_name='Brain_Region_From', value_name='SC_mean')
                       .assign(Base_Region_From = lambda x: x['Brain_Region_From'].str.replace("L_", "").str.replace("R_", ""),
                                   Base_Region_To = lambda x: x['Brain_Region_To'].str.replace("L_", "").str.replace("R_", ""),
                                   Hemi_From = lambda x: np.where(x['Brain_Region_From'].str.startswith("L_"), "Left", "Right"),
                                   Hemi_To = lambda x: np.where(x['Brain_Region_To'].str.startswith("L_"), "Left", "Right"))
                                   .query("Base_Region_From == Base_Region_To")
                                   .groupby(["Base_Region_From"])['SC_mean']
                                   .mean() 
                                   .reset_index()
                                   .rename(columns={"Base_Region_From": "Base_Region"})
                     .assign(SC_mean = lambda x: np.log10(x.SC_mean))
                     # Set values to 0 if they are negative or -Inf
                     .assign(SC_mean = lambda x: np.where(x.SC_mean < 0, 0, x.SC_mean))

                      )

/Users/abry4213/miniforge3/envs/annie_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
%%R -i fallon2020_SC_data

fallon2020_SC_data %>%
    mutate(label = glue("lh_{Base_Region}")) %>%
    left_join(., as_tibble(dk)) %>%
    ggseg(atlas = dk, mapping = aes(fill = SC_mean),
            position = "stacked", colour = "black", hemisphere="left",
            linewidth=0.5) +
    theme_void() +
    labs(fill = "Log-streamline count") +
    scale_fill_gradient(low="white", high="#10b8e8", na.value="white")+
    theme(plot.title = element_blank(),
    legend.key.width  = unit(3, "lines"),
    legend.key.height  = unit(0.75, "lines"),
    legend.position = "bottom") +
    guides(fill = guide_colourbar(frame.colour = "black", ticks.colour = "black",
                                  title.position="top", title.hjust=0.5))
# ggsave("../plots/brain_maps_corrs/SC_Fallon_HCP100_nothresh_mean_log_ggseg.svg", width=4, height=2, dpi=300, units='in')

Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'


In [12]:
merged_fallonSC_enigmaFC = pd.merge(fallon2020_SC_data, pearson_homotopic_res_mean, on='Base_Region', how='left')

In [25]:
%%R -i merged_fallonSC_enigmaFC

merged_fallonSC_enigmaFC %>%
    ggplot(data=., mapping=aes(x=SC_mean, y=Mean_Pearson_R, fill=SC_mean)) +
    geom_point(color='black', shape=21, size=4) +
    scale_fill_gradient(low="white", high="#10b8e8", na.value="white")+
    # ggpubr::stat_cor(method="spearman", cor.coef.name="rho", size=5, aes(label = ..r.label..)) +
    theme(legend.position='none')

ggsave("../plots/brain_maps_corrs/SC_Fallon_HCP100_nothresh_mean_log_scatter.svg", width=3.25, height=2.85, dpi=300, units="in")

# Spin test

In [15]:
# Define the left- and right-hemisphere Desikan-Killiany (DK) atlas
lh_dk = "../data/fsaverage/lh.aparc.annot"
rh_dk = "../data/fsaverage/rh.aparc.annot"

# Convert the DK atlas to gifti format
# dk_gifti = neuromaps.images.annot_to_gifti((lh_dk, rh_dk), background=None)
dk_gifti = neuromaps.images.annot_to_gifti(lh_dk, background=None)
dk_gifti = neuromaps.images.relabel_gifti(dk_gifti)

# Fit a Parcellator object
dk_parc = Parcellater(dk_gifti, 'fsLR', resampling_target='parcellation', hemi='L')

# Create a dictionary of values to compare
rng = np.random.default_rng(seed=127)  # for reproducibility
struct_measure = merged_fallonSC_enigmaFC.SC_mean.values
HoFC_measure = merged_fallonSC_enigmaFC['Mean_Pearson_R'].values

# Assume you are working with fsaverage and the Desikan-Killiany atlas
atlas = 'fsLR'

# Use the corresponding brain map to generate the nulls
spins = nulls.alexander_bloch(struct_measure, atlas=atlas, parcellation=dk_gifti, density='164k',
                            n_perm=10000, seed=127)

# Step 4: Compare HoFC and corresponding brain map using the nulls
stat, p = spearmanr(HoFC_measure, struct_measure)
stat_spin, p_spin = stats.compare_images(HoFC_measure, struct_measure, ignore_zero=False, nulls=spins, metric='spearmanr')

# this_map
this_map_spin_res = pd.DataFrame({"Comparison": "Fallon_HCP100_SC", 
                                    "SpearmanR": stat, 
                                    "pval": p,
                                    "SpearmanR_spin": stat_spin,
                                    "pval_spin": p_spin}, index=[0])
this_map_spin_res

,Comparison,SpearmanR,pval,SpearmanR_spin,pval_spin
0,Fallon_HCP100_SC,0.176758,0.3173,0.176758,0.313469


fallon2020_SC_data.merge(pearson_homotopic_res_mean, on='Base_Region', how='left') 